In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

data = """
[
  {
    "name": "Jane Smith",
    "age": 28,
    "purchases": [
      {
        "product_id": "B001",
        "product_name": "Tablet",
        "price": 399.99,
        "quantity": 1,
        "purchase_date": "2023-06-20"
      },
      {
        "product_id": "B002",
        "product_name": "Smartwatch",
        "price": 249.99,
        "quantity": 1,
        "purchase_date": "2023-07-05"
      }
    ]
  },
  {
    "name": "Alice Johnson",
    "age": 35,
    "purchases": [
      {
        "product_id": "C001",
        "product_name": "Bluetooth Speaker",
        "price": 149.99,
        "quantity": 1,
        "purchase_date": "2023-08-10"
      },
      {
        "product_id": "C002",
        "product_name": "Camera",
        "price": 199.99,
        "quantity": 1,
        "purchase_date": "2023-08-25"
      },
      {
        "product_id": "C003",
        "product_name": "VR Headset",
        "price": 299.99,
        "quantity": 1,
        "purchase_date": "2023-09-12"
      }
    ]
  },
  {
    "name": "John Doe",
    "age": 30,
    "purchases": [
      {
        "product_id": "A001",
        "product_name": "Smartphone",
        "price": 699.99,
        "quantity": 1,
        "purchase_date": "2023-07-15"
      },
      {
        "product_id": "A002",
        "product_name": "Laptop",
        "price": 999.99,
        "quantity": 1,
        "purchase_date": "2023-08-02"
      },
      {
        "product_id": "A003",
        "product_name": "Wireless Headphones",
        "price": 199.99,
        "quantity": 2,
        "purchase_date": "2023-09-10"
      }
    ]
  }
]
"""

In [4]:
spark = SparkSession.Builder().appName('json_data').getOrCreate()

24/11/12 09:18:41 WARN Utils: Your hostname, codebase resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/11/12 09:18:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 09:18:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
schema = StructType([
    StructField('name', StringType()),
    StructField('age', IntegerType()),
    StructField('purchases', ArrayType(
        StructType([
            StructField('product_id', StringType()),
            StructField('product_name', StringType()),
            StructField('price', DoubleType()),
            StructField('quantity', IntegerType()),
            StructField('purchase_date', StringType()),
        ])
    )),
])

In [29]:
df = spark.read.json(spark.sparkContext.parallelize([data]), schema = schema, multiLine=True)
df.show()

+-------------+---+--------------------+
|         name|age|           purchases|
+-------------+---+--------------------+
|   Jane Smith| 28|[{B001, Tablet, 3...|
|Alice Johnson| 35|[{C001, Bluetooth...|
|     John Doe| 30|[{A001, Smartphon...|
+-------------+---+--------------------+



In [30]:
df_exploded = df.withColumn('purchase', explode('purchases'))
df_exploded = df_exploded.select(
    "*",
    "purchase.product_id",
    "purchase.product_name",
    "purchase.price",
    "purchase.quantity",
    "purchase.purchase_date"
)
df_exploded.show()

+-------------+---+--------------------+--------------------+----------+-------------------+------+--------+-------------+
|         name|age|           purchases|            purchase|product_id|       product_name| price|quantity|purchase_date|
+-------------+---+--------------------+--------------------+----------+-------------------+------+--------+-------------+
|   Jane Smith| 28|[{B001, Tablet, 3...|{B001, Tablet, 39...|      B001|             Tablet|399.99|       1|   2023-06-20|
|   Jane Smith| 28|[{B001, Tablet, 3...|{B002, Smartwatch...|      B002|         Smartwatch|249.99|       1|   2023-07-05|
|Alice Johnson| 35|[{C001, Bluetooth...|{C001, Bluetooth ...|      C001|  Bluetooth Speaker|149.99|       1|   2023-08-10|
|Alice Johnson| 35|[{C001, Bluetooth...|{C002, Camera, 19...|      C002|             Camera|199.99|       1|   2023-08-25|
|Alice Johnson| 35|[{C001, Bluetooth...|{C003, VR Headset...|      C003|         VR Headset|299.99|       1|   2023-09-12|
|     John Doe| 

In [32]:
spark.stop()

In [33]:
!pwd

/home/de-ninja/Documents/Interview-Prep/coditas
